##**TEAM:**
* Uma Maheshwari [C0928443]
* Jharana Adhikari [C0927442]
* Yasaswin Palukuri [C0928450]
* Raemil Corniel [C0922799]


##**Topic**

###Credit Card Fraud Detection

####**Dataset Details:**

Source: https://www.kaggle.com/datasets/mishra5001/credit-card/data

**I. About:**
- Get insights of the loan defaultes
- Get insights into customer spending behavior and loan defaulters.
- Identify patterns in credit usage, payments, and purchases.
- Perform customer segmentation to group similar spending profiles.
- Help financial institutions optimize marketing strategies and risk management.

**II. Problem statement**
- Financial institutions need to understand customer spending and repayment behavior to provide better credit offerings.
- The dataset helps in identifying potential defaulters by analyzing credit usage patterns.
- Dimensionality reduction (PCA) and clustering techniques can uncover hidden patterns in customer transactions.
- The goal is to segment customers for personalized financial products and risk assessment.

**III. Dataset Information**









### **Assignment**


Part I
1. Sample a subset of the data based on class label
2. Apply PCA and LDA and visualize the results
3. Perform classification using naïve Bayes and interpret the results
4. Perform classification using Logistic Regression and interpret the results
5. Perform classification using SVMs and interpret the results

6. Comment on which approaches might allow you to best identify the confidence of class assignments (e.g. how happy, or how sad rather than just a label)
7. Plot results based on the approach in f.

Part II
1. Apply PCA and LDA. For PCA, plot the incremental gain in capturing the variance in the data for each successive principal component that is added and compare this with the subset of data in step 1.
2. Perform classification using naïve Bayes and interpret and visualize the results
3. Perform classification using Logistic Regression and interpret and visualize the results
4. Perform classification using SVMs and interpret and visualize the results
5. Discuss which classes are most similar / different with some justification given
6. Explain how you might identify faces that are mislabeled in the dataset and provide an implementation.

* Optional

Based on a 7-dimensional vector of confidence values for face identity, plot the position of these in a 2D embedding. The “points” that are shown should be the actual face images within the plot.



# 1. Import libraries


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Load data

In [5]:
application_data = pd.read_csv('application_data.csv')
previous_application = pd.read_csv('previous_application.csv')


In [6]:
application_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


# 3. Data Exploration Analysis

In [10]:

print(f"application_data shape: {application_data.shape}")
print(f"previous_application shape: {previous_application.shape}")

application_data shape: (307511, 122)
previous_application shape: (1670214, 37)


In [9]:
#identify null values for the application_dataset

application_data.isnull().sum()

,0
SK_ID_CURR,0
TARGET,0
NAME_CONTRACT_TYPE,0
CODE_GENDER,0
FLAG_OWN_CAR,0
...,...
AMT_REQ_CREDIT_BUREAU_DAY,41519
AMT_REQ_CREDIT_BUREAU_WEEK,41519
AMT_REQ_CREDIT_BUREAU_MON,41519
AMT_REQ_CREDIT_BUREAU_QRT,41519


In [11]:
#identify null values for the previous_application_dataset

previous_application.isnull().sum()

,0
SK_ID_PREV,0
SK_ID_CURR,0
NAME_CONTRACT_TYPE,0
AMT_ANNUITY,372235
AMT_APPLICATION,0
AMT_CREDIT,1
AMT_DOWN_PAYMENT,895844
AMT_GOODS_PRICE,385515
WEEKDAY_APPR_PROCESS_START,0
HOUR_APPR_PROCESS_START,0


In [12]:
previous_application['NAME_CONTRACT_TYPE'].unique()


array(['Consumer loans', 'Cash loans', 'Revolving loans', 'XNA'],
      dtype=object)

In [13]:
previous_application['NAME_CONTRACT_TYPE'].value_counts()


,count
NAME_CONTRACT_TYPE,
Cash loans,747553
Consumer loans,729151
Revolving loans,193164
XNA,346




> NAME_CONTRACT_TYPE shows the type of loan the client has or had at the current time. Since all datasets are related to a loan, this column does not add meaningful value to our process. This column will be dropped, and XNA rows will be deleted since it is a loan type non-identified.

Also the following columns does not add value to our dataset since are customer personal information or data related to time

WEEKDAY_APPR_PROCESS_START
HOUR_APPR_PROCESS_START
FLAG_LAST_APPL_PER_CONTRACT
NFLAG_LAST_APPL_IN_DAY
NFLAG_MICRO_CASH

NAME_CASH_LOAN_PURPOSE
NAME_CONTRACT_STATUS
DAYS_DECISION
NAME_PAYMENT_TYPE

NAME_TYPE_SUITE
NAME_CLIENT_TYPE
NAME_GOODS_CATEGORY
NAME_PORTFOLIO
NAME_PRODUCT_TYPE
CHANNEL_TYPE
SELLERPLACE_AREA
NAME_SELLER_INDUSTRY
CNT_PAYMENT
NAME_YIELD_GROUP
PRODUCT_COMBINATION
DAYS_FIRST_DRAWING
DAYS_FIRST_DUE
DAYS_LAST_DUE_1ST_VERSION
DAYS_LAST_DUE
DAYS_TERMINATION
NFLAG_INSURED_ON_APPROVAL




> Finally our previous application dataset will contain only


SK_ID_PREV
SK_ID_CURR

AMT_ANNUITY
AMT_APPLICATION
AMT_CREDIT
AMT_DOWN_PAYMENT
AMT_GOODS_PRICE

RATE_DOWN_PAYMENT
RATE_INTEREST_PRIMARY
RATE_INTEREST_PRIVILEGED
CODE_REJECT_REASON






In [15]:
columns_to_keep_previous_application = [
    'SK_ID_PREV', 'SK_ID_CURR',
    'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
    'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'CODE_REJECT_REASON'
]

#show columns to keep
previous_application[columns_to_keep_previous_application]



,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,CODE_REJECT_REASON
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,0.000000,0.182832,0.867336,XAP
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,NaN,NaN,NaN,XAP
2,2523466,122040,15060.735,112500.0,136444.5,NaN,112500.0,NaN,NaN,NaN,XAP
3,2819243,176158,47041.335,450000.0,470790.0,NaN,450000.0,NaN,NaN,NaN,XAP
4,1784265,202054,31924.395,337500.0,404055.0,NaN,337500.0,NaN,NaN,NaN,HC
...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,14704.290,267295.5,311400.0,0.0,267295.5,0.000000,NaN,NaN,XAP
1670210,2357031,334635,6622.020,87750.0,64291.5,29250.0,87750.0,0.340554,NaN,NaN,XAP
1670211,2659632,249544,11520.855,105237.0,102523.5,10525.5,105237.0,0.101401,NaN,NaN,XAP
1670212,2785582,400317,18821.520,180000.0,191880.0,NaN,180000.0,NaN,NaN,NaN,XAP


Let's check the application_data to identify the null values, and columns that might be relevant for our work

In [16]:
application_data.isnull().sum()

,0
SK_ID_CURR,0
TARGET,0
NAME_CONTRACT_TYPE,0
CODE_GENDER,0
FLAG_OWN_CAR,0
...,...
AMT_REQ_CREDIT_BUREAU_DAY,41519
AMT_REQ_CREDIT_BUREAU_WEEK,41519
AMT_REQ_CREDIT_BUREAU_MON,41519
AMT_REQ_CREDIT_BUREAU_QRT,41519


In [18]:
# Ensure all columns are displayed
pd.set_option('display.max_columns', None)

# Show dataset info
application_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB




> After analyzing each column and the meaning of its value we notice a big chunk is related to personal information like:


NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
REGION_POPULATION_RELATIVE
DAYS_BIRTH



> which is not relevant for us at the moment since we already have the previous application of the user, therefore we can use part of that dataset along with a few columns of the application dataset, which are related a relevant info like amount of properties owned, days employed, target column which will help us identify if default or not.

Columns to keep

SK_ID_CURR
TARGET
NAME_CONTRACT_TYPE

FLAG_OWN_CAR
FLAG_OWN_REALTY
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE

DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH

AMT_REQ_CREDIT_BUREAU_HOUR
AMT_REQ_CREDIT_BUREAU_DAY
AMT_REQ_CREDIT_BUREAU_WEEK
AMT_REQ_CREDIT_BUREAU_MON
AMT_REQ_CREDIT_BUREAU_QRT
AMT_REQ_CREDIT_BUREAU_YEAR




In [20]:
columns_to_keep_application = [
    'SK_ID_CURR',
    'TARGET',
    'FLAG_OWN_CAR',
    'FLAG_OWN_REALTY',
    'CNT_CHILDREN',
    'AMT_INCOME_TOTAL',
    'AMT_CREDIT',
    'AMT_ANNUITY',
    'AMT_GOODS_PRICE',
    'DAYS_EMPLOYED',
    'DAYS_REGISTRATION',
    'DAYS_ID_PUBLISH',
    'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY',
    'AMT_REQ_CREDIT_BUREAU_WEEK',
    'AMT_REQ_CREDIT_BUREAU_MON',
    'AMT_REQ_CREDIT_BUREAU_QRT',
    'AMT_REQ_CREDIT_BUREAU_YEAR'
]


application_data[columns_to_keep_application]

,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,N,Y,0,202500.0,406597.5,24700.5,351000.0,-637,-3648.0,-2120,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,N,N,0,270000.0,1293502.5,35698.5,1129500.0,-1188,-1186.0,-291,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Y,Y,0,67500.0,135000.0,6750.0,135000.0,-225,-4260.0,-2531,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,N,Y,0,135000.0,312682.5,29686.5,297000.0,-3039,-9833.0,-2437,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,N,Y,0,121500.0,513000.0,21865.5,513000.0,-3038,-4311.0,-3458,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,N,N,0,157500.0,254700.0,27558.0,225000.0,-236,-8456.0,-1982,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,N,Y,0,72000.0,269550.0,12001.5,225000.0,365243,-4388.0,-4090,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,N,Y,0,153000.0,677664.0,29979.0,585000.0,-7921,-6737.0,-5150,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,N,Y,0,171000.0,370107.0,20205.0,319500.0,-4786,-2562.0,-931,0.0,0.0,0.0,0.0,0.0,0.0


## I.Visualize the data to gain more insights

In [ ]:

# Merge datasets on 'SK_ID_CURR'
df_merged = application_data.merge(previous_application, on='SK_ID_CURR', how='left')

# Display first few rows
print(df_merged.head())

# Check the shape of the merged dataset
print(f"Original application_data shape: {application_data.shape}")
print(f"Original previous_application shape: {previous_application.shape}")
print(f"Merged dataset shape: {df_merged.shape}")